In [141]:
# IMPORTING LIBRARIES 
import pandas as pd
import sklearn.preprocessing
from sklearn.neural_network import MLPClassifier 

In [142]:
# starting with these parameters
n = 3 
profit_taking = 0.05
p = 0.60

In [143]:
# READ IN DATA
df_second_ten = pd.read_csv("data/EUR_GBP_2015_2005.csv")
df_first_ten = pd.read_csv("data/EUR_GBP_2025_2015.csv")

df = pd.concat([df_second_ten, df_first_ten], ignore_index = True)

df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%y")

df = df.sort_values(by =  "Date", ascending=True)

df = df.rename(columns = {'Price': 'Close'})
df = df[['Date', 'Close', 'Open', 'High', 'Low', 'Change %']]
df = df[['Date', 'Close', 'Open', 'High', 'Low', 'Change %']].copy()
df.head()

,Date,Close,Open,High,Low,Change %
2629,2005-09-16,0.6765,0.6768,0.6790,0.6746,0.00%
2628,2005-09-19,0.6735,0.6747,0.6759,0.6723,-0.44%
2627,2005-09-20,0.6736,0.6734,0.6753,0.6725,0.01%
2626,2005-09-21,0.6750,0.6737,0.6763,0.6727,0.21%
2625,2005-09-22,0.6786,0.6750,0.6800,0.6743,0.53%


In [144]:
df["target"] = (df["High"] > (profit_taking+1)*df["Open"]).astype(int)

In [145]:
# CREATE PRIOR n DAYS FEATURE

for before in range(1, n+1):
    df[f"Close_{before}_before"] = df["Close"].shift(before)
    df[f"Open_{before}_before"] = df["Open"].shift(before)
    df[f"High_{before}_before"] = df["High"].shift(before)
    df[f"Low_{before}_before"] = df["Low"].shift(before)

df = df.dropna()
df = df.drop(columns = ["Date", "Change %", "Close", "Open", "High", "Low"])
df.head()

,target,Close_1_before,Open_1_before,High_1_before,Low_1_before,Close_2_before,Open_2_before,High_2_before,Low_2_before,Close_3_before,Open_3_before,High_3_before,Low_3_before
2626,0,0.6736,0.6734,0.6753,0.6725,0.6735,0.6747,0.6759,0.6723,0.6765,0.6768,0.6790,0.6746
2625,0,0.6750,0.6737,0.6763,0.6727,0.6736,0.6734,0.6753,0.6725,0.6735,0.6747,0.6759,0.6723
2624,0,0.6786,0.6750,0.6800,0.6743,0.6750,0.6737,0.6763,0.6727,0.6736,0.6734,0.6753,0.6725
2623,0,0.6778,0.6788,0.6801,0.6769,0.6786,0.6750,0.6800,0.6743,0.6750,0.6737,0.6763,0.6727
2622,0,0.6787,0.6791,0.6801,0.6774,0.6778,0.6788,0.6801,0.6769,0.6786,0.6750,0.6800,0.6743


In [146]:
target = df[["target"]]
df = df.drop(columns = "target")

print(target.shape)
print(df.shape)
print(target.value_counts())

(5259, 1)
(5259, 12)
target
0         5257
1            2
Name: count, dtype: int64


In [147]:
# CREATING TRAINING, VALIDATION, and TESTING SETS
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# ensure that the ratios sum to 1.0
epsilon = 1e-10
assert(1 - epsilon <= train_ratio + validation_ratio + test_ratio <= 1 + epsilon)

x_train0, x_test, y_train0, y_test = sklearn.model_selection.train_test_split(
    df,
    target,
    test_size=test_ratio,
    random_state=42,
    )
print(f"len(x_train0)={len(x_train0)}")
print(f"len(x_test)={len(x_test)}")

x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(
    x_train0,
    y_train0,
    test_size=validation_ratio/(train_ratio + validation_ratio),
    random_state=0,
    )
print(f"len(x_train)={len(x_train)}")
print(f"len(x_val)={len(x_val)}")

len(x_train0)=4733
len(x_test)=526
len(x_train)=3944
len(x_val)=789


In [148]:
# MODEL TRAINING
model = sklearn.neural_network.MLPClassifier(
    hidden_layer_sizes=[10, 10],
    activation='logistic',
    alpha=1e-6,
    max_iter=1000,
    tol=1e-6,
    verbose=1,
    )

model.fit(x_train, y_train)

validation_accuracy = model.score(x_val, y_val)
print(f"validation_accuracy={validation_accuracy:0.4f}")
train_accuracy = model.score(x_train, y_train)
print(f"train_accuracy={train_accuracy:0.4f}")

Iteration 1, loss = 0.83306815
Iteration 2, loss = 0.72602721
Iteration 3, loss = 0.63334091
Iteration 4, loss = 0.55296750
Iteration 5, loss = 0.48422882
Iteration 6, loss = 0.42636812
Iteration 7, loss = 0.37809377
Iteration 8, loss = 0.33783733
Iteration 9, loss = 0.30402920
Iteration 10, loss = 0.27537061
Iteration 11, loss = 0.25076265
Iteration 12, loss = 0.22943541
Iteration 13, loss = 0.21081556
Iteration 14, loss = 0.19446588
Iteration 15, loss = 0.18002286
Iteration 16, loss = 0.16723558
Iteration 17, loss = 0.15584286
Iteration 18, loss = 0.14566684
Iteration 19, loss = 0.13652490
Iteration 20, loss = 0.12826320
Iteration 21, loss = 0.12075738
Iteration 22, loss = 0.11393081
Iteration 23, loss = 0.10766505
Iteration 24, loss = 0.10188198
Iteration 25, loss = 0.09655077
Iteration 26, loss = 0.09158141
Iteration 27, loss = 0.08694676
Iteration 28, loss = 0.08260095
Iteration 29, loss = 0.07850507
Iteration 30, loss = 0.07462648
Iteration 31, loss = 0.07095278
Iteration 32, los

/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 71, loss = 0.01601178
Iteration 72, loss = 0.01566620
Iteration 73, loss = 0.01533565
Iteration 74, loss = 0.01501594
Iteration 75, loss = 0.01470481
Iteration 76, loss = 0.01441510
Iteration 77, loss = 0.01413274
Iteration 78, loss = 0.01385839
Iteration 79, loss = 0.01359399
Iteration 80, loss = 0.01334246
Iteration 81, loss = 0.01310011
Iteration 82, loss = 0.01286205
Iteration 83, loss = 0.01263604
Iteration 84, loss = 0.01241730
Iteration 85, loss = 0.01220686
Iteration 86, loss = 0.01200131
Iteration 87, loss = 0.01180459
Iteration 88, loss = 0.01161392
Iteration 89, loss = 0.01143048
Iteration 90, loss = 0.01125296
Iteration 91, loss = 0.01108035
Iteration 92, loss = 0.01091306
Iteration 93, loss = 0.01075185
Iteration 94, loss = 0.01059642
Iteration 95, loss = 0.01044427
Iteration 96, loss = 0.01029715
Iteration 97, loss = 0.01015377
Iteration 98, loss = 0.01001714
Iteration 99, loss = 0.00988431
Iteration 100, loss = 0.00975421
Iteration 101, loss = 0.00962522
Iterat

In [149]:
if True:
    model.fit(x_train0, y_train0)
    test_accuracy = model.score(x_test, y_test)
    print(f"test_accuracy={test_accuracy}")

Iteration 1, loss = 0.74735211
Iteration 2, loss = 0.64378800
Iteration 3, loss = 0.55663866
Iteration 4, loss = 0.48321196
Iteration 5, loss = 0.42029648
Iteration 6, loss = 0.36514659
Iteration 7, loss = 0.31596078
Iteration 8, loss = 0.27191090
Iteration 9, loss = 0.23296055
Iteration 10, loss = 0.19921494
Iteration 11, loss = 0.17062030
Iteration 12, loss = 0.14687329
Iteration 13, loss = 0.12734291
Iteration 14, loss = 0.11141485
Iteration 15, loss = 0.09837386
Iteration 16, loss = 0.08761073
Iteration 17, loss = 0.07869473
Iteration 18, loss = 0.07120613
Iteration 19, loss = 0.06485000
Iteration 20, loss = 0.05942182


/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 21, loss = 0.05472121
Iteration 22, loss = 0.05063856
Iteration 23, loss = 0.04704752
Iteration 24, loss = 0.04387524
Iteration 25, loss = 0.04106474
Iteration 26, loss = 0.03855342
Iteration 27, loss = 0.03628773
Iteration 28, loss = 0.03425367
Iteration 29, loss = 0.03240820
Iteration 30, loss = 0.03072888
Iteration 31, loss = 0.02919178
Iteration 32, loss = 0.02778209
Iteration 33, loss = 0.02649159
Iteration 34, loss = 0.02530244
Iteration 35, loss = 0.02420762
Iteration 36, loss = 0.02318326
Iteration 37, loss = 0.02224336
Iteration 38, loss = 0.02136343
Iteration 39, loss = 0.02054398
Iteration 40, loss = 0.01978297
Iteration 41, loss = 0.01906739
Iteration 42, loss = 0.01839848
Iteration 43, loss = 0.01777138
Iteration 44, loss = 0.01718166
Iteration 45, loss = 0.01662762
Iteration 46, loss = 0.01610750
Iteration 47, loss = 0.01560887
Iteration 48, loss = 0.01514592
Iteration 49, loss = 0.01470259
Iteration 50, loss = 0.01428488
Iteration 51, loss = 0.01388954
Iteratio

In [150]:
y_true = y_test.values.ravel()
probs_test = model.predict_proba(x_test)[:, 1]
preds_test = model.predict(x_test)

In [151]:
print(probs_test.shape)
print(preds_test.shape)

(526,)
(526,)


In [152]:
print(probs_test)

[0.00048434 0.00048496 0.00048387 0.0004864  0.0004851  0.00048721
 0.00048688 0.00048727 0.00048458 0.00048391 0.0004872  0.00048592
 0.00048706 0.00048386 0.00048477 0.00048432 0.00048403 0.00048403
 0.00048393 0.0004847  0.00048449 0.00048448 0.00048416 0.00048514
 0.0004849  0.00048731 0.00048426 0.0004837  0.00048459 0.00048373
 0.00048376 0.00048445 0.0004871  0.00048397 0.00048732 0.00048404
 0.00048366 0.00048372 0.00048409 0.00048382 0.00048371 0.0004839
 0.00048416 0.00048396 0.00048348 0.00048396 0.00048368 0.00048455
 0.00048723 0.00048391 0.00048396 0.00048434 0.00048423 0.00048397
 0.00048481 0.00048395 0.00048408 0.0004842  0.00048393 0.00048603
 0.0004838  0.00048448 0.00048401 0.00048416 0.00048605 0.00048438
 0.00048423 0.00048507 0.00048505 0.00048394 0.00048426 0.00048616
 0.00048584 0.00048353 0.00048411 0.00048419 0.00048377 0.00048735
 0.00048432 0.0004839  0.00048779 0.00048492 0.00048505 0.00048369
 0.00048418 0.00048403 0.00048365 0.00048444 0.00048387 0.00048